# 🔧 Notebook 4: Ordinal Models
## Final Project - Ordinal vs Nominal Sentiment Analysis
### Atharv Chaudhary

---

**Purpose:** Train and evaluate ORDINAL classification models.

**Models:**
1. Ridge Regression (treats ratings as continuous)
2. Ordinal Logistic Regression (threshold-based)

**Input:** `amazon_electronics_cleaned.csv`

**Output:** `ordinal_results.csv`, confusion matrices

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import (
    accuracy_score, 
    mean_absolute_error, 
    f1_score, 
    classification_report,
    confusion_matrix
)

# Settings
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
plt.style.use('seaborn-v0_8-whitegrid')

print("✅ Libraries imported")

## Step 1: Load Data & Prepare Features

In [ ]:
# Load cleaned data
df = pd.read_csv('amazon_electronics_cleaned.csv')
print(f"✅ Loaded {len(df):,} reviews")

# TF-IDF
vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.95
)

X = vectorizer.fit_transform(df['text'])
y = df['rating'].values

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"✅ Training: {X_train.shape[0]:,} | Test: {X_test.shape[0]:,}")

## Step 2: Helper Functions

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    """Evaluate model and return metrics."""
    accuracy = accuracy_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    
    print(f"\n{'='*55}")
    print(f"📊 {model_name}")
    print(f"{'='*55}")
    print(f"Accuracy:      {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"MAE:           {mae:.4f}")
    print(f"F1 (macro):    {f1_macro:.4f}")
    print(f"F1 (weighted): {f1_weighted:.4f}")
    
    return {
        'model': model_name,
        'encoding': 'Ordinal',
        'accuracy': accuracy,
        'mae': mae,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }


def calculate_error_rates(y_true, y_pred):
    """Calculate adjacent and severe error rates."""
    errors = y_true != y_pred
    if errors.sum() == 0:
        return 0.0, 0.0
    
    error_distances = np.abs(y_true[errors] - y_pred[errors])
    adjacent = (error_distances == 1).sum() / errors.sum()
    severe = (error_distances >= 2).sum() / errors.sum()
    
    return adjacent, severe


print("✅ Helper functions defined")

## Step 3: Model 1 - Ridge Regression (Ordinal)

In [ ]:
# ============================================================================
# MODEL 1: RIDGE REGRESSION (ORDINAL)
# ============================================================================

print("\n" + "="*70)
print("🔧 MODEL 1: Ridge Regression (Ordinal)")
print("="*70)
print("\nTreats ratings as CONTINUOUS ordinal values: 1 < 2 < 3 < 4 < 5")
print("Formula: ŷ = wᵀx + b, then round to nearest integer")
print("Loss: Σ(yᵢ - ŷᵢ)² + λ||w||² (naturally penalizes large errors more)")

# Train
ridge_model = Ridge(alpha=1.0, random_state=RANDOM_STATE)
ridge_model.fit(X_train, y_train)

# Predict (continuous) then round and clip
ridge_pred_continuous = ridge_model.predict(X_test)
ridge_pred = np.clip(np.round(ridge_pred_continuous), 1, 5).astype(int)

# Evaluate
ridge_results = evaluate_model(y_test, ridge_pred, "Ridge Regression")

# Error analysis
ridge_adjacent, ridge_severe = calculate_error_rates(y_test, ridge_pred)
ridge_results['adjacent_error'] = ridge_adjacent
ridge_results['severe_error'] = ridge_severe

print(f"\nError Analysis:")
print(f"   Adjacent Error Rate (±1): {ridge_adjacent:.2%}")
print(f"   Severe Error Rate (±2+):  {ridge_severe:.2%}")

In [ ]:
# Show continuous prediction distribution
print("\n📊 Ridge Continuous Predictions:")
print(f"   Min: {ridge_pred_continuous.min():.2f}")
print(f"   Max: {ridge_pred_continuous.max():.2f}")
print(f"   Mean: {ridge_pred_continuous.mean():.2f}")

# Classification report
print("\n📋 Classification Report - Ridge:")
print(classification_report(y_test, ridge_pred, digits=4))

## Step 4: Model 2 - Ordinal Logistic Regression

In [ ]:
# ============================================================================
# ORDINAL LOGISTIC REGRESSION CLASS
# ============================================================================

class OrdinalLogisticRegression:
    """
    Ordinal Logistic Regression using threshold (cumulative) approach.
    
    For K classes, trains K-1 binary classifiers.
    Each classifier k models: P(Y ≤ k | x) = σ(θₖ - wᵀx)
    
    This EXPLICITLY preserves ordinal structure: 1 < 2 < 3 < 4 < 5
    
    Key insight: Same feature weights w for all thresholds,
    only the threshold θₖ changes.
    """
    
    def __init__(self, max_iter=1000, C=1.0):
        self.max_iter = max_iter
        self.C = C
        self.classifiers = []
        self.classes_ = None
    
    def fit(self, X, y):
        self.classes_ = np.sort(np.unique(y))
        n_classes = len(self.classes_)
        
        print(f"   Training {n_classes - 1} binary classifiers...")
        print(f"   Classes: {self.classes_}")
        
        # Train K-1 binary classifiers for cumulative probabilities
        for k in range(n_classes - 1):
            threshold = self.classes_[k]
            # Binary target: 1 if y <= threshold, 0 otherwise
            y_binary = (y <= threshold).astype(int)
            
            clf = LogisticRegression(
                max_iter=self.max_iter, 
                C=self.C,
                random_state=42,
                solver='lbfgs'
            )
            clf.fit(X, y_binary)
            self.classifiers.append(clf)
            
            pos_rate = y_binary.mean()
            print(f"      Classifier {k+1}: P(Y ≤ {threshold}) - {pos_rate:.1%} positive")
        
        return self
    
    def predict_proba(self, X):
        """Predict class probabilities using cumulative approach."""
        n_samples = X.shape[0]
        n_classes = len(self.classes_)
        
        # Get cumulative probabilities P(Y ≤ k)
        cumulative_probs = np.zeros((n_samples, n_classes))
        cumulative_probs[:, -1] = 1.0  # P(Y ≤ max_class) = 1
        
        for k, clf in enumerate(self.classifiers):
            cumulative_probs[:, k] = clf.predict_proba(X)[:, 1]
        
        # Convert cumulative to class probabilities: P(Y = k) = P(Y ≤ k) - P(Y ≤ k-1)
        class_probs = np.zeros((n_samples, n_classes))
        class_probs[:, 0] = cumulative_probs[:, 0]  # P(Y = 1) = P(Y ≤ 1)
        
        for k in range(1, n_classes):
            class_probs[:, k] = cumulative_probs[:, k] - cumulative_probs[:, k-1]
        
        # Ensure non-negative (numerical stability)
        class_probs = np.maximum(class_probs, 0)
        
        # Normalize rows to sum to 1
        row_sums = class_probs.sum(axis=1, keepdims=True)
        row_sums[row_sums == 0] = 1  # Avoid division by zero
        class_probs = class_probs / row_sums
        
        return class_probs
    
    def predict(self, X):
        """Predict class labels."""
        probs = self.predict_proba(X)
        return self.classes_[np.argmax(probs, axis=1)]


print("✅ OrdinalLogisticRegression class defined")

In [ ]:
# ============================================================================
# MODEL 2: ORDINAL LOGISTIC REGRESSION
# ============================================================================

print("\n" + "="*70)
print("🔧 MODEL 2: Ordinal Logistic Regression (Threshold-Based)")
print("="*70)
print("\nUses K-1 binary classifiers to model cumulative probabilities.")
print("Formula: P(Y ≤ k | x) = σ(θₖ - wᵀx)")
print("Class prob: P(Y = k) = P(Y ≤ k) - P(Y ≤ k-1)")
print("\nExplicitly encodes ordinal constraint: θ₁ < θ₂ < θ₃ < θ₄\n")

# Train
olr_model = OrdinalLogisticRegression(max_iter=1000, C=1.0)
olr_model.fit(X_train, y_train)

# Predict
print("\n   Making predictions...")
olr_pred = olr_model.predict(X_test)

# Evaluate
olr_results = evaluate_model(y_test, olr_pred, "Ordinal Logistic Regression")

# Error analysis
olr_adjacent, olr_severe = calculate_error_rates(y_test, olr_pred)
olr_results['adjacent_error'] = olr_adjacent
olr_results['severe_error'] = olr_severe

print(f"\nError Analysis:")
print(f"   Adjacent Error Rate (±1): {olr_adjacent:.2%}")
print(f"   Severe Error Rate (±2+):  {olr_severe:.2%}")

In [ ]:
# Classification report
print("\n📋 Classification Report - Ordinal LR:")
print(classification_report(y_test, olr_pred, digits=4))

## Step 5: Confusion Matrices

In [ ]:
# ============================================================================
# CONFUSION MATRICES
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Ridge Regression
cm_ridge = confusion_matrix(y_test, ridge_pred)
sns.heatmap(cm_ridge, annot=True, fmt='d', cmap='Oranges', ax=axes[0],
            xticklabels=[1, 2, 3, 4, 5], yticklabels=[1, 2, 3, 4, 5])
axes[0].set_xlabel('Predicted Rating', fontsize=11)
axes[0].set_ylabel('Actual Rating', fontsize=11)
axes[0].set_title(f'Ridge Regression (Ordinal)\nAccuracy: {ridge_results["accuracy"]:.2%}', 
                  fontsize=12, fontweight='bold')

# Ordinal Logistic Regression
cm_olr = confusion_matrix(y_test, olr_pred)
sns.heatmap(cm_olr, annot=True, fmt='d', cmap='Purples', ax=axes[1],
            xticklabels=[1, 2, 3, 4, 5], yticklabels=[1, 2, 3, 4, 5])
axes[1].set_xlabel('Predicted Rating', fontsize=11)
axes[1].set_ylabel('Actual Rating', fontsize=11)
axes[1].set_title(f'Ordinal Logistic Regression\nAccuracy: {olr_results["accuracy"]:.2%}', 
                  fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('confusion_matrices_ordinal.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Saved: confusion_matrices_ordinal.png")

## Step 6: Save Results

In [ ]:
# ============================================================================
# SAVE RESULTS
# ============================================================================

# Combine results
ordinal_results = pd.DataFrame([ridge_results, olr_results])

print("\n" + "="*70)
print("📊 ORDINAL MODELS SUMMARY")
print("="*70)
print(ordinal_results.to_string(index=False))

# Save to CSV
ordinal_results.to_csv('ordinal_results.csv', index=False)
print("\n✅ Saved: ordinal_results.csv")

In [ ]:
# Save predictions
predictions_df = pd.DataFrame({
    'actual': y_test,
    'ridge_pred': ridge_pred,
    'olr_pred': olr_pred
})
predictions_df.to_csv('ordinal_predictions.csv', index=False)
print("✅ Saved: ordinal_predictions.csv")

In [ ]:
# Download files
try:
    from google.colab import files
    files.download('ordinal_results.csv')
    files.download('confusion_matrices_ordinal.png')
except:
    print("Files saved locally")

---
## ✅ Summary

**Ordinal Models Trained:**

| Model | Accuracy | MAE | Adjacent Error | Severe Error |
|-------|----------|-----|----------------|---------------|
| Ridge Regression | See above | See above | See above | See above |
| Ordinal LR | See above | See above | See above | See above |

**Key Insight:** Ordinal models should have LOWER severe error rates because they understand that 1→5 is worse than 4→5.

**Next:** Run `5_Results_Analysis.ipynb` to compare all models.